Created on Tue Apr 21 11:10:11 2020

@author: Deeps

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install torch
!pip install torchsummary
! pip install torchvision

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm # Displays a progress bar

import torch
from torch.utils import data
from torchsummary import summary
from torch import nn
from torch import optim
from PIL import Image
from sklearn.model_selection import train_test_split as tts
import torch.nn.functional as F
from torchvision import datasets,models, transforms
from torch.utils.data import Dataset, Subset, DataLoader, random_split
from data_gen import Dataset
from augment import Augmentations as aug
from visualization  import visualizer 

## Load the dataset and train, val, test splits

In [3]:
num_epochs = 50
num_classes = 2
batch_size = 4
learning_rate = 0.001
datacsv = pd.read_csv("data.csv")

cv_results = []

### ------------Data Augmentation--------

In [4]:
data_base_aug = dict(datacsv)
data_base_aug["image"] = list(data_base_aug["image"])
data_base_aug["label"] = list(data_base_aug["label"])

rot = aug()
for k in range(len(data_base_aug["image"])):
    result = rot.rotate_append(data_base_aug["image"][k],data_base_aug["label"][k])
    datacsv = datacsv.append(result,ignore_index = True)

_____

# ----------Create dataset generator-------------

In [5]:
for cv in range(1):
    x_train,x_test =tts(datacsv["image"],test_size=1/6, shuffle=True)
    partition = {'train':list(x_train),'validation':list(x_test)}
    labels = {}
    
    for k in range(len(datacsv['image'])):
        labels['%s'%datacsv['image'][k]] = datacsv['label'][k]
        
    #####################################################################
    
    training_set = Dataset(partition["train"],labels)
    training_generator = data.DataLoader(training_set, batch_size=batch_size, shuffle=True)
    
    validation_set = Dataset(partition['validation'], labels)
    validation_generator = data.DataLoader(validation_set, batch_size=batch_size, shuffle=True)
    
    #######################################################
    class Network(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(3, 6, 5)
            self.conv2 = nn.Conv2d(6, 6, 5)
            self.conv3 = nn.Conv2d(6, 3, (5,4))
            self.conv4 = nn.Conv2d(3, 3, (7,4))
            self.pool = nn.MaxPool2d(2, 2)
            self.poolav = nn.AvgPool2d(2,2)
            self.fc1 = nn.Linear(3* 42 * 29, 128)
            self.fc2 = nn.Linear(128, num_classes)
            
    
        def forward(self,x):
            # TODO: Design your own network, implement forward pass here
            x = self.pool(F.relu(self.conv1(x))) #3*748*500 -> 6*744*496 -> 6*372*2
            x = self.poolav(F.relu(self.conv2(x))) #6*372*254 -> 12*368*250 -> 12*184*125
            x = self.pool(F.relu(self.conv3(x))) #12*184*125 -> 6*180*122 -> 6*90*61
            x = self.poolav(F.relu(self.conv4(x))) #6*90*61-> 3*84*58 -> 3*42*29
            #print("Before view: %s by %s"%(x.shape[-2],x.shape[-1]))
            x = x.view(-1, 3 * 42 * 29)
            x = F.relu(self.fc1(x))
            out = F.softmax(self.fc2(x))
            return out
    
    
    device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device
    model = Network().to(device)
    summary(model,(3,748,512))
    criterion = nn.CrossEntropyLoss() # Specify the loss layer
    optimizer = optim.Adam(model.parameters(),lr=0.001) # Specify optimizer and assign trainable parameters to it, weight_decay is L2 regularization strength
    
    
    
    def train(model, training_generator, num_epoch = num_epochs): # Train the model
        print("Start training...")
        model.train() # Set the model to training mode
        for i in range(num_epoch):
            running_loss = []
            accuracy = []
            for batch, label in tqdm(training_generator):
                batch = batch.to(device)
                label = label.to(device)
                optimizer.zero_grad() # Clear gradients from the previous iteration
                pred = model(batch) # This will call Network.forward() that you implement
                loss = criterion(pred, label) # Calculate the loss
                running_loss.append(loss.item())
                correct = (torch.argmax(pred,dim=1)==label).sum().item()
                accuracy.append(correct/batch_size)
                loss.backward() # Backprop gradients to all tensors in the network
                optimizer.step() # Update trainable weights
            print("Epoch {} loss:{}".format(i+1,np.mean(running_loss))) # Print the average loss for this epoch
            print("Epoch {} Accuracy:{}".format(i+1,np.mean(accuracy)))
        print("Done!")
        try:
            torch.save(model, "Trained_model/teeth_model.pth")
            print("Model saved!")
        except:
            print("Could not save model")
    
    def evaluate(model, validation_generator): # Evaluate accuracy on validation / test set
        model.eval() # Set the model to evaluation mode
        correct = 0
        total = 0
        with torch.set_grad_enabled(True): # Do not calculate grident to speed up computation
            for batch, label in tqdm(validation_generator):
                batch = batch.to(device)
                label = label.to(device)
                pred = model(batch)
                correct += (torch.argmax(pred,dim=1)==label).sum().item()
                total+=batch_size
        acc = correct/total
        print("Evaluation accuracy: {}".format(acc))
        return acc
        
    train(model, training_generator, num_epochs)
    print("Evaluate on validation set...")
    result = evaluate(model, validation_generator)
    cv_results.append(result)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 6, 744, 508]             456
         MaxPool2d-2          [-1, 6, 372, 254]               0
            Conv2d-3          [-1, 6, 368, 250]             906
         AvgPool2d-4          [-1, 6, 184, 125]               0
            Conv2d-5          [-1, 3, 180, 122]             363
         MaxPool2d-6            [-1, 3, 90, 61]               0
            Conv2d-7            [-1, 3, 84, 58]             255
         AvgPool2d-8            [-1, 3, 42, 29]               0
            Linear-9                  [-1, 128]         467,840
           Linear-10                    [-1, 2]             258
Total params: 470,078
Trainable params: 470,078
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 4.38
Forward/backward pass size (MB): 27.66
Params size (MB): 1.79
Estimated 

RuntimeError: shape '[-1, 3654]' is invalid for input of size 13932

In [ ]:
print("Mean accuracy: %s"%(np.mean(cv_results)*100))

In [ ]:
vis=visualizer()
vis.plot_filter(layer=model.conv1,single_channel = False)

In [ ]:
vis.plot_filter(layer=model.conv2,single_channel = False)

In [ ]:
vis.plot_filter(layer=model.conv3,single_channel = False)

In [ ]:
vis.plot_filter(layer=model.conv4,single_channel = False)

In [ ]:
vis.plot_filter(layer=model.fc1,single_channel = False)